# Fuga de datos
Encuentre y solucione este problema que arruina su modelo de manera sutil.

En este tutorial, aprenderá qué es la fuga de datos y cómo prevenirla. Si no sabe cómo prevenirlo, aparecerán fugas con frecuencia y arruinarán sus modelos de maneras sutiles y peligrosas. Entonces, este es uno de los conceptos más importantes para los científicos de datos en ejercicio.

### Introducción
La fuga de datos (o fuga) ocurre cuando sus datos de entrenamiento contienen información sobre el objetivo, pero datos similares no estarán disponibles cuando el modelo se use para la predicción. Esto conduce a un alto rendimiento en el conjunto de entrenamiento (y posiblemente incluso en los datos de validación), pero el modelo tendrá un rendimiento deficiente en producción.

En otras palabras, la fuga hace que un modelo parezca preciso hasta que se empieza a tomar decisiones con el modelo, y luego el modelo se vuelve muy inexacto.

Hay dos tipos principales de fugas: fugas objetivo y contaminación de prueba de tren.

#### Fuga objetivo
La fuga de objetivos se produce cuando sus predictores incluyen datos que no estarán disponibles en el momento de realizar las predicciones. Es importante pensar en la fuga de objetivos en términos del momento o el orden cronológico en que los datos están disponibles, no simplemente si una característica ayuda a hacer buenas predicciones.

Un ejemplo será útil. Imagine que quiere predecir quién enfermará de neumonía. Las primeras filas de sus datos sin procesar se ven así:

Las personas toman antibióticos después de contraer neumonía para recuperarse. Los datos sin procesar muestran una fuerte relación entre esas columnas, pero take_antibiotic_medicine se cambia con frecuencia después de que se determina el valor de got_pneumonia. Esta es una fuga de objetivo.

El modelo vería que cualquiera que tenga un valor Falso para tomó_antibiótico_medicina no tenía neumonía. Dado que los datos de validación provienen de la misma fuente que los datos de entrenamiento, el patrón se repetirá en la validación y el modelo tendrá excelentes puntuaciones de validación (o validación cruzada).

Pero el modelo será muy inexacto cuando se implemente posteriormente en el mundo real, porque incluso los pacientes que contraerán neumonía aún no habrán recibido antibióticos cuando necesitemos hacer predicciones sobre su salud futura.

Para evitar este tipo de fuga de datos, se debe excluir cualquier variable actualizada (o creada) después de alcanzar el valor objetivo.

#### Contaminación de prueba de tren
Se produce un tipo diferente de fuga cuando no se tiene cuidado de distinguir los datos de entrenamiento de los datos de validación.

Recuerde que la validación pretende ser una medida de cómo funciona el modelo con datos que no ha considerado antes. Puede corromper este proceso de manera sutil si los datos de validación afectan el comportamiento de preprocesamiento. A esto a veces se le llama contaminación de prueba de tren.

Por ejemplo, imagina que ejecutas un preprocesamiento (como ajustar un imputador para los valores faltantes) antes de llamar a train_test_split(). ¿El final resulto? Su modelo puede obtener buenas puntuaciones de validación, lo que le dará una gran confianza en él, pero tendrá un rendimiento deficiente cuando lo implemente para tomar decisiones.

Después de todo, usted incorporó datos de la validación o los datos de prueba en la forma en que hace predicciones, por lo que pueden funcionar bien con esos datos en particular, incluso si no se pueden generalizar a datos nuevos. Este problema se vuelve aún más sutil (y más peligroso) cuando realiza ingeniería de funciones más compleja.

Si su validación se basa en una división simple de prueba de tren, excluya los datos de validación de cualquier tipo de ajuste, incluido el ajuste de pasos de preprocesamiento. Esto es más fácil si utiliza canalizaciones de scikit-learn. Cuando se utiliza validación cruzada, es aún más importante que realice el preprocesamiento dentro del proceso.

### Ejemplo
En este ejemplo, aprenderá una forma de detectar y eliminar fugas de objetivos.

Usaremos un conjunto de datos sobre solicitudes de tarjetas de crédito y omitiremos el código de configuración de datos básicos. El resultado final es que la información sobre cada solicitud de tarjeta de crédito se almacena en un DataFrame X. Lo usaremos para predecir qué solicitudes fueron aceptadas en una Serie y.

In [1]:
import pandas as pd

# Read the data
data = pd.read_csv('AER_credit_card_data.csv', 
                   true_values = ['yes'], false_values = ['no'])

# Select target
y = data.card

# Select predictors
X = data.drop(['card'], axis=1)

print("Number of rows in the dataset:", X.shape[0])
X.head()

Number of rows in the dataset: 1319


,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,0,37.66667,4.5200,0.033270,124.983300,True,False,3,54,1,12
1,0,33.25000,2.4200,0.005217,9.854167,False,False,3,34,1,13
2,0,33.66667,4.5000,0.004156,15.000000,True,False,4,58,1,5
3,0,30.50000,2.5400,0.065214,137.869200,False,False,0,25,1,7
4,0,32.16667,9.7867,0.067051,546.503300,True,False,2,64,1,5


Dado que se trata de un conjunto de datos pequeño, utilizaremos validación cruzada para garantizar medidas precisas de la calidad del modelo.

In [2]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Since there is no preprocessing, we don't need a pipeline (used anyway as best practice!)
my_pipeline = make_pipeline(RandomForestClassifier(n_estimators=100))
cv_scores = cross_val_score(my_pipeline, X, y, 
                            cv=5,
                            scoring='accuracy')

print("Cross-validation accuracy: %f" % cv_scores.mean())

Cross-validation accuracy: 0.979531


Con experiencia, descubrirá que es muy raro encontrar modelos que sean precisos el 98% de las veces. Sucede, pero es bastante poco común como para que debamos inspeccionar los datos más de cerca para detectar fugas de objetivos.

Aquí hay un resumen de los datos, que también puede encontrar en la pestaña de datos:

tarjeta: 1 si se acepta la solicitud de tarjeta de crédito, 0 si no

informes: Número de informes despectivos importantes

edad: Edad n años más doceavos de año

Ingresos: Ingresos anuales (divididos por 10,000)

Participación: Relación entre el gasto mensual en tarjetas de crédito y los ingresos anuales.

gasto: Gasto promedio mensual con tarjeta de crédito

propietario: 1 si es propietario de la casa, 0 si alquila

selfempl: 1 si trabaja por cuenta propia, 0 si no

dependientes: 1 + número de dependientes

meses: meses viviendo en la dirección actual

majorcards: Número de tarjetas de crédito principales poseídas

activo: Número de cuentas de crédito activas

Algunas variables parecen sospechosas. Por ejemplo, ¿gasto significa gasto en esta tarjeta o en tarjetas utilizadas antes de presentar la solicitud?

En este punto, las comparaciones de datos básicos pueden resultar muy útiles:

In [3]:
expenditures_cardholders = X.expenditure[y]
expenditures_noncardholders = X.expenditure[~y]

print('Fraction of those who did not receive a card and had no expenditures: %.2f' \
      %((expenditures_noncardholders == 0).mean()))
print('Fraction of those who received a card and had no expenditures: %.2f' \
      %(( expenditures_cardholders == 0).mean()))

Fraction of those who did not receive a card and had no expenditures: 1.00
Fraction of those who received a card and had no expenditures: 0.02


Como se muestra arriba, todos los que no recibieron una tarjeta no tuvieron gastos, mientras que sólo el 2% de los que recibieron una tarjeta no tuvieron gastos. No es sorprendente que nuestro modelo pareciera tener una alta precisión. Pero este también parece ser un caso de fuga de objetivos, donde los gastos probablemente significan gastos en la tarjeta que solicitaron.

Dado que la participación está parcialmente determinada por el gasto, también debería excluirse. Las variables activas y cartas principales son un poco menos claras, pero por la descripción suenan preocupantes. En la mayoría de las situaciones, es mejor prevenir que curar si no puedes localizar a las personas que crearon los datos para obtener más información.

Ejecutaríamos un modelo sin fuga objetivo de la siguiente manera:

In [4]:
# Drop leaky predictors from dataset
potential_leaks = ['expenditure', 'share', 'active', 'majorcards']
X2 = X.drop(potential_leaks, axis=1)

# Evaluate the model with leaky predictors removed
cv_scores = cross_val_score(my_pipeline, X2, y, 
                            cv=5,
                            scoring='accuracy')

print("Cross-val accuracy: %f" % cv_scores.mean())

Cross-val accuracy: 0.833198


Esta precisión es bastante menor, lo que puede resultar decepcionante. Sin embargo, podemos esperar que sea correcto aproximadamente el 80% de las veces cuando se use en aplicaciones nuevas, mientras que el modelo con fugas probablemente funcionaría mucho peor que eso (a pesar de su puntuación aparente más alta en la validación cruzada).

### Conclusión
La fuga de datos puede suponer un error multimillonario en muchas aplicaciones de ciencia de datos. Una separación cuidadosa de los datos de capacitación y validación puede prevenir la contaminación de las pruebas de trenes, y las tuberías pueden ayudar a implementar esta separación. Asimismo, una combinación de precaución, sentido común y exploración de datos puede ayudar a identificar las fugas objetivo.

# EJERCICIOS

### Paso 1: La ciencia de datos de los cordones de los zapatos

Nike lo contrató como consultor de ciencia de datos para ayudarlos a ahorrar dinero en materiales de calzado. Su primera tarea es revisar un modelo que uno de sus empleados construyó para predecir cuántos cordones necesitarán cada mes. Las características del modelo de aprendizaje automático incluyen:

El mes actual (enero, febrero, etc.)

Gastos en publicidad en el mes anterior.

Varias características macroeconómicas (como la tasa de desempleo) a principios del mes actual

La cantidad de cuero que terminaron usando en el mes actual.

Los resultados muestran que el modelo es casi perfectamente preciso si se incluye la característica sobre la cantidad de cuero que usaron. Pero es sólo moderadamente preciso si omite esa característica. Te das cuenta de que esto se debe a que la cantidad de cuero que utilizan es un indicador perfecto de cuántos zapatos producen, lo que a su vez te indica cuántos cordones necesitan.

¿Cree que la característica de cuero usado constituye una fuente de fuga de datos? Si tu respuesta es "depende", ¿de qué depende?

Después de haber pensado en su respuesta, compárela con la solución a continuación.

Esto es complicado y depende de los detalles de cómo se recopilan los datos (lo cual es común cuando se piensa en fugas). ¿Decidirías a principios de mes cuánto cuero se utilizará ese mes? Si es así, está bien. Pero si eso se determina durante el mes, no tendrás acceso a él cuando hagas la predicción. Si tiene una estimación al comienzo del mes y luego la cambia durante el mes, la cantidad real utilizada durante el mes no se puede utilizar como característica (porque causa fugas).

### Paso 2: Devolución de los cordones
Tienes una nueva idea. Podrías usar la cantidad de cuero que Nike pidió (en lugar de la cantidad que realmente usaron) hasta un mes determinado como predictor en tu modelo de cordones.

¿Cambia esto su respuesta sobre si hay un problema de fuga? Si respondes "depende", ¿de qué depende?

Esto podría estar bien, pero depende de si piden primero los cordones o el cuero. Si piden cordones primero, no sabrás cuánto cuero han pedido cuando predigas sus necesidades de cordones. Si piden cuero primero, tendrá ese número disponible cuando realice su pedido de cordones y todo estará bien.

Paso 3: ¿Hacerse rico con las criptomonedas?
Le ahorraste tanto dinero a Nike que te dieron un bono. Felicidades.

Su amigo, que también es científico de datos, dice que ha creado un modelo que le permitirá convertir su bonificación en millones de dólares. Específicamente, su modelo predice el precio de una nueva criptomoneda (como Bitcoin, pero más nueva) un día antes del momento de la predicción. Su plan es comprar la criptomoneda cada vez que el modelo diga que el precio de la moneda (en dólares) está a punto de subir.

Las características más importantes de su modelo son:

Precio actual de la moneda.

Cantidad de moneda vendida en las últimas 24 horas

Cambio en el precio de la moneda en las últimas 24 horas.

Cambio en el precio de la moneda en la última hora.

Número de tweets nuevos en las últimas 24 horas que mencionan la moneda

El valor de la criptomoneda en dólares ha fluctuado hacia arriba y hacia abajo en más de $
100 en el último año y, sin embargo, el error promedio de su modelo es inferior a $1. Dice que esto es una prueba de que su modelo es preciso y que usted debe invertir con él, comprando la moneda cada vez que el modelo diga que está a punto de subir.

¿Tiene razón? Si hay un problema con su modelo, ¿cuál es?

Aquí no hay ninguna fuente de fuga. Estas funciones deberían estar disponibles en el momento en que desee realizar una predicción y es poco probable que se cambien en los datos de entrenamiento después de que se determine el objetivo de predicción. Pero la forma en que describe la precisión puede ser engañosa si no se tiene cuidado. Si el precio se mueve gradualmente, el precio de hoy será un predictor preciso del precio de mañana, pero es posible que no le diga si es un buen momento para invertir. Por ejemplo, si es 100𝑡𝑜𝑑𝑎𝑦,𝑎𝑚𝑜𝑑𝑒𝑙𝑝𝑟𝑒𝑑𝑖𝑐𝑡𝑖𝑛𝑔𝑎𝑝𝑟𝑖𝑐𝑒 𝑜𝑓 100 mañana puede parecer exacto, incluso si no puede indicarle si el precio sube o baja con respecto al precio actual. Un mejor objetivo de predicción sería el cambio de precio durante el día siguiente. Si puede predecir consistentemente si el precio está a punto de subir o bajar (y en qué medida), es posible que tenga una oportunidad de inversión ganadora.

### Paso 4: Prevención de infecciones
Una agencia que brinda atención médica quiere predecir qué pacientes de una cirugía poco común están en riesgo de infección, de modo que pueda alertar a las enfermeras para que tengan especial cuidado al realizar un seguimiento de esos pacientes.

Quieres construir un modelo. Cada fila en el conjunto de datos de modelado será un único paciente que recibió la cirugía, y el objetivo de predicción será si contrajo una infección.

Algunos cirujanos pueden realizar el procedimiento de una manera que aumente o reduzca el riesgo de infección. Pero, ¿cuál es la mejor manera de incorporar la información del cirujano al modelo?

Tienes una idea inteligente.

Tome todas las cirugías de cada cirujano y calcule la tasa de infección entre esos cirujanos.

Para cada paciente en los datos, averigüe quién fue el cirujano e ingrese la tasa de infección promedio de ese cirujano como una característica.

¿Esto plantea algún problema de fuga de objetivos? ¿Plantea algún problema de contaminación durante las pruebas de trenes?

Esto plantea un riesgo tanto de fuga del objetivo como de contaminación del tren de prueba (aunque es posible que pueda evitar ambas cosas si tiene cuidado).

Tiene una fuga objetivo si el resultado de un paciente determinado contribuye a la tasa de infección de su cirujano, que luego se vuelve a conectar al modelo de predicción para determinar si ese paciente se infecta. Puede evitar la fuga de objetivos si calcula la tasa de infección del cirujano utilizando solo las cirugías antes del paciente que estamos prediciendo. Calcular esto para cada cirugía en tus datos de entrenamiento puede ser un poco complicado.

También tiene un problema de contaminación de la prueba del tren si lo calcula utilizando todas las cirugías realizadas por un cirujano, incluidas las del conjunto de prueba. El resultado sería que su modelo podría parecer muy preciso en el conjunto de prueba, incluso si no se generalizaría bien a nuevos pacientes una vez implementado el modelo. Esto sucedería porque la función de riesgo del cirujano tiene en cuenta los datos del conjunto de prueba. Existen conjuntos de pruebas para estimar cómo funcionará el modelo al ver nuevos datos. Entonces esta contaminación anula el propósito del equipo de prueba.

### Paso 5: Precios de la vivienda
Construirás un modelo para predecir los precios de la vivienda. El modelo se implementará de forma continua para predecir el precio de una casa nueva cuando se agregue una descripción a un sitio web. Aquí hay cuatro características que podrían usarse como predictores.

Tamaño de la casa (en metros cuadrados)

Precio medio de venta de viviendas del mismo barrio

Latitud y longitud de la casa.

Si la casa tiene sótano

Tienes datos históricos para entrenar y validar el modelo.
¿Cuál de las características es más probable que sea una fuente de fuga?

In [ ]:
# Fill in the line below with one of 1, 2, 3 or 4.
potential_leakage_feature = 2

2 es la fuente de la fuga objetivo. A continuación se muestra un análisis de cada característica:

Es poco probable que el tamaño de una casa cambie después de su venta (aunque técnicamente es posible). Pero normalmente esto estará disponible cuando necesitemos hacer una predicción y los datos no se modificarán después de que se venda la casa. Entonces es bastante seguro.

No conocemos las reglas para cuando esto se actualice. Si el campo se actualiza en los datos sin procesar después de que se vendió una casa y la venta de la casa se utiliza para calcular el promedio, esto constituye un caso de fuga objetivo. En un caso extremo, si sólo se vende una casa en el vecindario y es la casa que estamos tratando de predecir, entonces el promedio será exactamente igual al valor que estamos tratando de predecir. En general, para barrios con pocas ventas, el modelo funcionará muy bien con los datos de entrenamiento. Pero cuando aplica el modelo, la casa que está pronosticando aún no se habrá vendido, por lo que esta característica no funcionará igual que en los datos de entrenamiento.

Estos no cambian y estarán disponibles en el momento en que queramos hacer una predicción. Así que aquí no hay riesgo de fuga de objetivos.

Esto tampoco cambia y está disponible en el momento que queremos hacer una predicción. Así que aquí no hay riesgo de fuga de objetivos.

### Conclusión

Las fugas son un problema difícil y sutil. Debería estar orgulloso si se dio cuenta de los problemas de estos ejemplos.

Ahora tiene las herramientas para crear modelos de alta precisión y abordar los problemas prácticos más difíciles que surgen al aplicar estos modelos para resolver problemas reales.

Todavía hay mucho espacio para desarrollar conocimientos y experiencia. Pruebe una competencia o consulte nuestros conjuntos de datos para practicar sus nuevas habilidades.